# 🛡️ Data Anonymization Pipeline (ETL)

**Nota para el lector:**
Este notebook contiene el código utilizado para procesar los archivos originales (`raw data`) proporcionados por la institución educativa.

El objetivo de este script es:
1. Ingestar los datos desde múltiples hojas de cálculo.
2. Eliminar columnas sensibles (PII - Información de Identificación Personal).
3. Generar identificadores sintéticos (`STD_XXX`) para proteger la identidad de los estudiantes.
4. Exportar el archivo `dataset_anonimizado_final.csv`.

⚠️ **Advertencia de Reproducibilidad:**
Dado que los archivos de entrada contienen datos reales de estudiantes, **no se encuentran incluidos en este repositorio público**. Por lo tanto, este notebook sirve únicamente como **evidencia del proceso de gobernanza de datos** y no podrá ser ejecutado sin los archivos fuente protegidos.

Para replicar el análisis, por favor diríjase al notebook `02_analysis_visualization.ipynb` que utiliza el dataset ya procesado y seguro.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
NOMBRE_ARCHIVO = "Matematica5toACE.xlsx"
# ==========================================
# 2. PROCESAMIENTO INTELIGENTE (ETL)
# ==========================================

def procesar_hoja(archivo_excel, nombre_hoja, nombre_seccion_destino):

    try:
        df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=None)
    except ValueError:
        print(f"⚠️ Alerta: No se encontró la pestaña '{nombre_hoja}'. Verifica el nombre en el Excel.")
        return pd.DataFrame() # Retorna vacío si falla

    # ---------------------------------------------------------
    # SELECCIÓN DE COLUMNAS (IGNORANDO PII)
    # ---------------------------------------------------------

    if df.shape[1] < 17:
        return pd.DataFrame()

    df_clean = df.iloc[8:, [2, 10, 16]].copy()
    df_clean.columns = ['Nombre_Original', 'Nota_Individual', 'Nota_Parejas']

    # Limpieza numérica
    df_clean['Nota_Individual'] = pd.to_numeric(df_clean['Nota_Individual'], errors='coerce')
    df_clean['Nota_Parejas'] = pd.to_numeric(df_clean['Nota_Parejas'], errors='coerce')

    # Eliminar filas vacías (donde no hay notas)
    df_clean = df_clean.dropna()

    df_clean['Seccion'] = nombre_seccion_destino
    return df_clean

# ---------------------------------------------------------
# CARGA DE DATOS
# ---------------------------------------------------------

df_A = procesar_hoja(NOMBRE_ARCHIVO, "5to. A", "A")
df_C = procesar_hoja(NOMBRE_ARCHIVO, "5to. C", "C")
df_E = procesar_hoja(NOMBRE_ARCHIVO, "5to. E", "E")

# Unificar todo
lista_dfs = [df for df in [df_A, df_C, df_E] if not df.empty]
if not lista_dfs:
    print("❌ Error: No se pudieron cargar datos. Revisa el nombre del archivo y las pestañas.")
else:
    df_total = pd.concat(lista_dfs, ignore_index=True)

    # ==========================================
    # 3. ANONIMIZACIÓN (PROTECCIÓN DE IDENTIDAD)
    # ==========================================
    # Creamos IDs como "STD_001", "STD_002"...
    df_total['Student_ID'] = ['STD_' + str(i+1).zfill(3) for i in range(len(df_total))]

    # Cálculos
    df_total['Mejora'] = df_total['Nota_Parejas'] - df_total['Nota_Individual']
    df_total['Categoria'] = np.where(df_total['Nota_Individual'] < 10, 'Bajo Rendimiento', 'Alto Rendimiento')

    # Seleccionamos SOLO las columnas seguras (Sin nombre, sin cédula)
    df_export = df_total[['Student_ID', 'Seccion', 'Nota_Individual', 'Nota_Parejas', 'Mejora', 'Categoria']].copy()

    print(f"✅ ¡Éxito! Dataset generado con {len(df_export)} estudiantes.")
    display(df_export.head())

    # Exportar
    df_export.to_csv('dataset_anonimizado_final.csv', index=False)
    print("📂 Archivo 'dataset_anonimizado_final.csv' listo para descargar.")

✅ ¡Éxito! Dataset generado con 72 estudiantes.


,Student_ID,Seccion,Nota_Individual,Nota_Parejas,Mejora,Categoria
0,STD_001,A,6,19,13,Bajo Rendimiento
1,STD_002,A,17,20,3,Alto Rendimiento
2,STD_003,A,20,20,0,Alto Rendimiento
3,STD_004,A,5,17,12,Bajo Rendimiento
4,STD_005,A,19,20,1,Alto Rendimiento


📂 Archivo 'dataset_anonimizado_final.csv' listo para descargar.
